In [1]:
import collections
import io
import time
import math
import pickle
import os
import pyndri
import pyndri.compat
import logging
import sys
import numpy as np
import gensim
import pandas as pd
from Helper import *

from copy import deepcopy
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn.model_selection import KFold

Trying to load statistics from file...Success!


/Users/andrei/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def load_pickle(fpath):
    with open(fpath, 'rb') as file:
        return pickle.load(file)


def save_pickle(obj, fpath):
    with open(fpath, 'wb') as file:
        pickle.dump(obj, file)

In [8]:
X_cols = ['TF-IDF', 'LDA', 'LSI', 'dp_mu_500', 'GLM_top1000docs_sigma50_mu1000', 'doc_len', 'query_len']
y_cols = ['relevance_label']

training_data = load_pickle('../pickles/LTR_DF_Training.pkl')
X_train = training_data[X_cols]
y_train = training_data[y_cols].values.ravel()
normalized_X_train = (X_train - X_train.mean()) / X_train.std()
normalized_X_train = normalized_X_train.values

validation_data = load_pickle('../pickles/LTR_DF_Validation.pkl')
X_validate = validation_data[X_cols]
y_validate = validation_data[y_cols].values.ravel()
normalized_X_validate = (X_validate - X_validate.mean()) / X_validate.std()
normalized_X_validate = normalized_X_validate.values

In [9]:
log_reg_cv = LogisticRegressionCV(cv=10, solver='lbfgs', penalty='l2')

In [10]:
log_reg_cv.fit(normalized_X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=10, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [14]:
results = log_reg_cv.predict(normalized_X_validate)

In [18]:
results

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [40]:
validation_data['relevance_result'] = results

In [41]:
validation_data.head()

,query_id,int_doc_id,ext_doc_id,relevance_label,TF-IDF,LDA,LSI,dp_mu_500,GLM_top1000docs_sigma50_mu1000,doc_len,query_len,results,result,relevance_result
69~AP880216-0127,69,94866,AP880216-0127,0,12.998833,0.808140,0.635558,-4.133619,-47.496657,440.0,5.0,0.0,0.0,0.0
69~AP880217-0029,69,130037,AP880217-0029,0,12.318767,0.765000,0.610453,-4.207090,-46.076589,283.0,5.0,0.0,0.0,0.0
69~AP880217-0156,69,130164,AP880217-0156,0,11.925768,0.792726,0.384661,-4.296502,-45.694837,317.0,5.0,0.0,0.0,0.0
69~AP880219-0069,69,157235,AP880219-0069,0,10.413368,0.840270,0.581214,-4.796670,-48.726499,364.0,5.0,0.0,0.0,0.0
69~AP880219-0139,69,157305,AP880219-0139,0,12.674277,0.820857,0.639198,-4.192220,-47.199983,354.0,5.0,0.0,0.0,0.0


In [42]:
validation_data.sort_values(by=['query_id', 'relevance_result'], ascending=[True, False], inplace=True)

In [28]:
data = collections.defaultdict(list)

for idx, row in validation_data.iterrows():
    data[row['query_id']].append((row['results'], row['ext_doc_id']))

In [31]:
with open('log_reg.run', 'w') as f_out:
    write_run(
        model_name='LogReg',
        data=data,
        out_f=f_out,
        max_objects_per_query=1000)